In [86]:
from jupyter_dash import JupyterDash
import dash_core_components as dcc
import dash_html_components as html
import pandas as pd
import plotly.express as px
from dash.dependencies import Input, Output


#
data = pd.read_csv('Datasets/classified_data1.csv',error_bad_lines=False, index_col=False, dtype='unicode', nrows=100000)
#data = data.query("state='Alabama'")

data.sort_values("likeCount", inplace = True)
data["likeCount"] = pd.to_numeric(data["likeCount"])
data["replyCount"] = pd.to_numeric(data["replyCount"])
data["retweetCount"] = pd.to_numeric(data["retweetCount"])
app = JupyterDash(__name__)
data["date"] = data["date"].astype('datetime64[ns]')


available_indicators = ["date","likeCount","replyCount","retweetCount","start date","sentiment"]
# add filter by state

minimum,maximum = 0, 3000



In [87]:


app.layout = html.Div([
    html.Div([
        html.Div([
            dcc.Dropdown(
                id="xaxis-column",
                options = [{'label': i, 'value': i} for i in available_indicators],
                value="date"
            ),
            #Radio item functionality not implemented yet
            dcc.RadioItems(
                id="xaxis-type",
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            
            )
            

        ]),
        
        html.Div([
            dcc.Dropdown(
                id="yaxis-column",
                options = [{'label': i, 'value': i} for i in available_indicators],
                value="likeCount"
            ),
            #Radio item functionality not implemented yet
            dcc.RadioItems(
                id="yaxis-type",
                options=[{'label': i, 'value': i} for i in ['Linear', 'Log']],
                value='Linear',
                labelStyle={'display': 'inline-block'}
            
            )
            

        ])
    ]),
    
    
    
    dcc.Graph(id="over-time-scatter"),
    html.P("Like Slider"),
    dcc.RangeSlider(
        id="x-range-slider",
        min=minimum, max = maximum, step = 1,
        marks={minimum: str(minimum), maximum: str(maximum)},
        value=[minimum,maximum]
    ),
    html.Div([
        "Min - x: ",
        dcc.Input(id="minimum-x-input", value=0, type='number',min=0),
        " Max - x: ",
        dcc.Input(id="maximum-x-input",value=100,type='number',min=0)]),
    
    html.Div([
        "Min - y: ",
        dcc.Input(id="minimum-y-input", value=0, type='number', min=0),
        " Max - y: ",
        dcc.Input(id="maximum-y-input", value=100,type='number', min=0)]),
    
    ]
)

@app.callback(
    Output("over-time-scatter", "figure"),
    Input('xaxis-column', 'value'),
    Input('yaxis-column', 'value'),
    Input('xaxis-type', 'value'),
    Input('yaxis-type', 'value'),
    [Input("x-range-slider", "value")],
    Input("minimum-x-input", "value"),
    Input("maximum-x-input", "value"),
    Input("minimum-y-input", "value"),
    Input("maximum-y-input", "value")
    

)
def update_graph(xaxis_column_name, yaxis_column_name,
                 xaxis_type, yaxis_type, slider_range,
                min_x, max_x,
                min_y,max_y):
    if xaxis_column_name == 'date':
        mask_x = (data["date"] >= data["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=min_x)) & (data["date"] <= data["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=max_x))
    else:
         mask_x = (data[xaxis_column_name] >= min_x) & (data[xaxis_column_name] <= max_x)
    if yaxis_column_name == 'date':
        mask_y = (data["date"] >= data["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=min_y)) & (data["date"] <= data["date"].min().replace(hour=0, minute=0, second=0) + pd.Timedelta(days=max_y))
    else:
        mask_y = (data[yaxis_column_name] >= min_y) & (data[yaxis_column_name] <= max_y)
    #mask is broken when using same data on both axis - not really a problem
    mask = mask_x & mask_y
    fig = px.scatter(
        data[mask], x=xaxis_column_name, y=yaxis_column_name)
    return fig


In [88]:
app.run_server(mode='inline', port=1002)

# 